In [1]:
# show the matplotlib plotted graph within notebook lines.
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### In case that your tqdm is not installed. Please go to the Start(Windows)->Anaconda2(64-bit)->Anaconda Prompt(py35)

### type: conda install tqdm -y

In [3]:
# import required libraries
# pandas:Data framework library for Python
# sklearn: Library to perform machine learning tasks
import os
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy
import scipy.stats as st
import re
import codecs
from collections import Counter
import nltk
from nltk.stem import *
from nltk.corpus import stopwords
import sklearn 
import sklearn.datasets
import sklearn.metrics as metrics 
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from scipy import stats

In [4]:
# extract data
#!gzip -d -k 20news-19997.tar.gz
#!tar -xf 20news-19997.tar

In [5]:
# display newsgroups directories
!ls 20_newsgroups

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


## Support functions

In [6]:
def getFilesDirectory(datapath='20_newsgroups'):
    # create file directory for all files
    files = []
    for (path, dirnames, filenames) in os.walk(datapath):
        files.extend(os.path.join(path, name) for name in filenames)
    # putting file directories into pandas dataframe
    directorydf= pd.DataFrame(files)
    directorydf.columns = ['Directories']
    
    return directorydf


In [100]:
directorydf = getFilesDirectory().head(n = 5000)
#directorydf



In [47]:
# create function to help encoding the targets
def label_target (d):
    if d.find("alt.atheism") > 0 :
      return 0
    if d.find("comp.graphics") > 0:
      return 1
    if d.find("comp.os.ms-windows.misc") > 0:
      return 2
    if d.find("comp.sys.ibm.pc.hardware") > 0:
      return 3
    if d.find("comp.sys.mac.hardware") > 0:
      return 4
    if d.find("comp.windows.x") > 0:
      return 5
    if d.find("misc.forsale") > 0:
      return 6
    if d.find("rec.autos") > 0:
      return 7
    if d.find("rec.motorcycles") > 0:
      return 8
    if d.find("rec.sport.baseball") > 0:
      return 9
    if d.find("rec.sport.hockey") > 0:
      return 10
    if d.find("sci.crypt") > 0:
      return 11
    if d.find("sci.electronics") > 0:
      return 12
    if d.find("sci.med") > 0:
      return 13
    if d.find("sci.space") > 0:
      return 14
    if d.find("soc.religion.christian") > 0:
      return 15
    if d.find("talk.politics.guns") > 0:
      return 16
    if d.find("talk.politics.mideast") > 0:
      return 17
    if d.find("talk.politics.misc") > 0:
      return 18
    if d.find("talk.religion.misc") > 0:
      return 19
    return 'Other'

## Q1

### (a)

i) The feature encoding method: stem, to exclude stop words
ii) The method for ranking features: MAP
iii) Number of features to select: 500, as we have 20000 documents, select a relative big number

### (b)

In [101]:
def dataPreprocessor(k, fs, encode, directorydf, size=None):
    """
    INPUT
        k: int. number of features to use
        fs: string. From ['tf', 'mi']
        encode: string. From ['tf', 'boolean']
        drectorydf: Dataframe. It is given, see above support functions
        size: int. Sample size. Default should be the data size.
    OUTPUT
        data: Dataframe. preprocessed data
    
    ps: 'tf' means term frequency, 'mi' means mutual information
    """

    ########### your code goes here ###########
    from sklearn.feature_selection import mutual_info_classif
    from whoosh.analysis import Filter
    from collections import Counter
    import re
    import codecs
    import numpy
    counter = Counter()
    
    wnLemm = WordNetLemmatizer()
    # Open the files and count the word frequency in each file in a loop and update the counter after finished processing a file
    for rownum, row in enumerate(directorydf.itertuples()):
        with codecs.open(row.Directories,"r" ,encoding='utf-8', errors='ignore') as myfile:
            counter.update([word for word in re.findall(r'\w+', wnLemm.lemmatize(myfile.read(), 'v'))])
        if (rownum % 1000 == 0):
            print("processed %d files" % (rownum+1))
    # filter words
    data = []
    feature_target = []
    topk = []
    if fs == 'tf':     
    # find top k words occured most requent in the document
        topk = counter.most_common(k)
    
    # find top k words with highest mi scores in the document
    elif fs =='mi':
        count = 0
        for key in dict(counter):
            count = count + 1
        words= counter.most_common(count)
        
        tempCounter = Counter()
        for rownum, row in enumerate(directorydf.itertuples()):
            with codecs.open(row.Directories,"r" ,encoding='utf-8', errors='ignore') as myfile:
                tempCounter= Counter([word for word in re.findall(r'\w+',wnLemm.lemmatize(myfile.read(), 'v'))])
                frequency = [tempCounter[word] if tempCounter[word] > 0 else 0 for (word,wordCount) in words] 
                feature_target.append(frequency+[label_target(row.Directories)])
    
        features = pd.DataFrame(feature_target)
        dfName = []
        for c in words:
            dfName.append(c[0])
        
        features.columns = dfName +['myTarget']
        features_df = features[dfName]
        feature = features_df.as_matrix()
        target_df = features['myTarget']
        target = target_df.as_matrix()
        mi = mutual_info_classif(feature, target)
        arr = numpy.array(mi)
        ind = arr.argsort()[-k:][::-1]
        for i in ind:
            topk.append((words[i][0],words[i][1]))
    
    np = []
    # now we had top k words, count the frequecy (binary) of these words in individual file
    for rownum, row in enumerate(directorydf.itertuples()):
        with codecs.open(row.Directories,"r" ,encoding='utf-8', errors='ignore') as myfile:
            tempCounter = Counter([word for word in re.findall(r'\w+',wnLemm.lemmatize(myfile.read(), 'v'))])
            if encode == 'boolean':
            # if the word appears in the doc, then 1, else 
                topkinDoc = [1 if tempCounter[word] > 0 else 0 for (word,wordCount) in topk]
            elif encode == 'tf':
                topkinDoc = [tempCounter[word] if tempCounter[word] > 0 else 0 for (word,wordCount) in topk]
        # create a list for top k words with encoded target and its label
            np.append(topkinDoc+[label_target(row.Directories)])
        if (rownum % 1000 == 0):
            print("processed %d files" % (rownum+1))
     
    data = pd.DataFrame(np)
    dfName = []
    for c in topk:
        dfName.append(c[0])
    data.columns = dfName +['myTarget']

    
    ###########         end         ###########%%
    return data

In [102]:
data = dataPreprocessor(k=500, fs='tf', encode='tf', directorydf=directorydf, size=None)


processed 1 files
processed 1001 files
processed 2001 files
processed 3001 files
processed 4001 files
processed 1 files
processed 1001 files
processed 2001 files
processed 3001 files
processed 4001 files


### (c)

In [31]:
#Confidence Interval Function
import scipy.stats
from math import sqrt
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    mu,sd = np.mean(a),np.std(a)
    z = stats.t.ppf(confidence, n)
    h=z*sd/sqrt(n)
    return mu, h

In [103]:
def randomSplitCI(data, clf, num_run, **params):
    """
    INPUT
        data: 2D numpy array. Pre-processed data
        clf: string. Name of the classifier from ['LR', 'SVM', 'NB']
        num_run: int. How many times you want to run for random evaluation?
        params: string->real. Hyper-parameter of classifier. PS: c=1.0, r=0.01
    
    OUTPUT
        train_scores: list. Results of trails
        test_scores: list. Results of trails
        train_mean: scalar. Average accuracy
        test_mean: scalar. Average accuracy
        train_ci: scalar. Confidence Interval
        test_ci: scalar. Confidence Interval
    """
    
    ########### your code goes here ###########
    import numpy as np
    features_df= data.iloc[:,0:-1]
    features = features_df.as_matrix()
    target = data['myTarget'].as_matrix()
    cms = {}
    scores_tr = []
    scores_ta = []
    cv = StratifiedKFold(target, n_folds=10)
    print(cv)
    for i, (train, test) in enumerate(cv):
        
        # train then immediately predict the test set
        #clfModel = clf.fit(features[train], target[train])
        #target_pred = clfModel.predict(features[test])
    
        if clf == 'LR':
            clf = LogisticRegression()

        # separate datasets into training and test datasets once, no folding
            #features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3)
        # train the features and target datasets and fit to a model
            clfModel = clf.fit(features[train], target[train])
        # predict target with feature test set using trained model
            target_pred = clfModel.predict(features[test])
        

    
        elif clf == 'SVM':
        # randomize and separate datasets into training and test datasets once, no folding
            features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)
            parameters = [{'kernel': ['linear'], 'C': [1]}]

        # find the best parameters to optimize accuracy
            svc_clf = SVC(C=1, probability= True)
            clfSVM = GridSearchCV(svc_clf, parameters, cv=10, scoring='accuracy') #5 folds
            clfSVM.fit(features_train, target_train) #train the model 
            print("Best parameters found from SVM's:")
            print(clfSVM.best_params_)
            print("Best score found from SVM's:")    
            print (clfSVM.best_score_)
        
        elif clf == 'NB':
            clf = GaussianNB()
            clfModel = clf.fit(features[train], target[train])
            target_pred = clfModel.predict(features[test])
            pd.DataFrame(metrics.confusion_matrix(target_test, target_predNB), columns=labels, index=labels)
        
        cms[i] = pd.DataFrame(metrics.confusion_matrix(target[test], target_pred), columns=target, index=target)
        scores_ta.append(metrics.accuracy_score(target[test], target_pred))
        #scores_tr.append(metrics.accuracy_score(target[train], features[test]))
   
    print(scores_ta)
    train_scores = np.mean(scores_ta)
    test_scores = [33]
    train_mean = 1.0
    test_mean = 1.0
    train_ci = 1.0
    test_ci = 1.0
    print("The accuracy score for this training is", test_scores)
    
    # compute the confusion matrix on each fold, convert it to a DataFrame and stash it for later compute
        #cms[i] = pandas.DataFrame(metrics.confusion_matrix(target[test], target_pred), columns=labels, index=labels)
    # stash the overall accuracy on the test set for the fold too
        #scores.append(metrics.accuracy_score(target[test], target_pred))
    ###########         end         ###########
    return train_scores,test_scores,train_mean,test_mean,train_ci,test_ci
    

In [104]:
randomSplitCI(data, 'LR', 1)

sklearn.cross_validation.StratifiedKFold(labels=[0 0 0 ..., 4 4 4], n_folds=10, shuffle=False, random_state=None)
[0.98399999999999999, 0.98399999999999999, 0.98399999999999999, 0.98399999999999999, 0.98399999999999999, 0.98399999999999999, 0.98399999999999999, 0.98399999999999999, 0.98399999999999999, 0.98399999999999999]
The accuracy score for this training is [33]


(0.98399999999999999, [33], 1.0, 1.0, 1.0, 1.0)

In [ ]:
train_scores,test_scores,train_mean,test_mean,train_ci,test_ci = randomSplitCI(data, 'LR', 10, c=1.0)
print("Train\
    \nResult of trails:{0} \
    \nAverage Accuracy: {1} \
    \nConfidence Interval: {2}\n".format(train_scores, train_mean, train_ci)
     )
print("Test\
    \nResult of trails:{0} \
    \nAverage Accuracy: {1} \
    \nConfidence Interval: {2}".format(test_scores, test_mean, test_ci)
     )

### (d)

### (e)

In [ ]:
def randomSplitCM(data, clf, num_run, **params):
    """
    INPUT
        data: Dataframe. Pre-processed data
        clf: string. Name of the classifier from ['LR', 'SVM', 'NB']
        params: string->real. Hyper-parameter of classifier. PS: c=1.0, r=0.01
    
    OUTPUT
        cm: pandas.DataFrame. Confusion Matrix
    """
    
    ########### your code goes here ###########

    
    
    
    
    
    ###########         end         ###########
    return cm

In [ ]:
cm = randomSplitCM(data, 'LR', 10, c=1.0)

### (f)

In [ ]:
sns.heatmap(cm)

## Q2

### (a)

In [ ]:
def featureSizeAC(data, clf, num_run, **params):
    """
    INPUT
        data: Dataframe. Pre-processed data
        clf: string. Name of the classifier from ['LR', 'SVM', 'NB']
        params: string->real. Hyper-parameter of classifier. PS: c=1.0, r=0.01
    
    OUTPUT
        train_mean_fs: list.
        train_ci_fs: list.
        test_mean_fs: list.
        test_ci_fs: list.
        
    """
    feature_precentage = np.linspace(0.1, 1, 10, endpoint=True)
    
    ########### your code goes here ###########


    
    
    
    ###########         end         ###########
    return train_mean_fs, train_ci_fs, test_mean_fs, test_ci_fs

In [ ]:
train_mean_fs, train_ci_fs, test_mean_fs, test_ci_fs = featureSizeAC(data, 'LR', 10, c=1.0)

### (b)

In [ ]:
def featureSizePlot(feature_size, train_mean_fs, train_ci_fs, test_mean_fs, test_ci_fs):
    # First illustrate basic pyplot interface, using defaults where possible.
    plt.figure()
    test_curve=plt.errorbar(feature_size, test_mean_fs, color=sns.xkcd_rgb["pale red"], yerr=test_ci_fs)
    train_curve=plt.errorbar(feature_size, train_mean_fs,color=sns.xkcd_rgb["denim blue"], yerr=train_ci_fs)
    plt.legend([test_curve, train_curve], ['Test', 'Train'])
    plt.xlabel('Feature Percentage')
    plt.ylabel('Accuracy')
    plt.title("Accuracy vs Feature Size")
    plt.show()

In [ ]:
featureSizePlot(np.linspace(0.1, 1, 10, endpoint=True),train_mean_fs, train_ci_fs, test_mean_fs, test_ci_fs )

## Q3

In [ ]:
def hyperParameterAC(data, clf, num_run):
    """
    INPUT
        data: Dataframe. Pre-processed data
        clf: string. Name of the classifier from ['LR', 'SVM', 'NB']
    
    OUTPUT
        test_mean_hp: list.  mean accuracy list of test
        test_ci_hp: list. confidence interval list of test
        train_mean_hp: list. mean accuracy list of train
        train_ci_hp: list. confidence interval list of train
        
    NOTE 
        randomSplitCI could be the sub-route of this function
    """
    params = np.logspace(-4, 4, num=9)
    
    ########### your code goes here ###########

    
    
    
    
    
    
    ###########         end         ###########
    return train_mean_hp, train_ci_hp, test_mean_hp, test_ci_hp

In [ ]:
train_mean_hp, train_ci_hp, test_mean_hp, test_ci_hp = hyperParameterAC(data, 'LR', 3)

In [ ]:
def hyperParameterPlot(params, train_mean_hp, train_ci_hp, test_mean_hp, test_ci_hp):
    # First illustrate basic pyplot interface, using defaults where possible.
    plt.figure()
    test_curve=plt.errorbar(params, test_mean_hp, color=sns.xkcd_rgb["pale red"], yerr=test_ci_hp)
    train_curve=plt.errorbar(params, train_mean_hp,color=sns.xkcd_rgb["denim blue"], yerr=train_ci_hp)
    plt.legend([test_curve, train_curve], ['Test', 'Train'])
    plt.xlabel('Parameter')
    plt.xscale("log")
    plt.ylabel('Accuracy')
    plt.title("Accuracy vs Parameters")
    plt.show()

In [ ]:
hyperParameterPlot(np.logspace(-4, 4, num=9),train_mean_hp, train_ci_hp, test_mean_hp, test_ci_hp )

## Q4

### (a)

In [ ]:
def dataSizeAC(data, clf, num_run):
    """
    INPUT
        data: Dataframe. Pre-processed data
        clf: string. Name of the classifier from ['LR', 'SVM', 'NB']
        params: string->real. Hyper-parameter of classifier. PS: c=1.0, r=0.01
    
    OUTPUT
        test_mean_ds: list.  mean accuracy list of test
        test_ci_ds: list. confidence interval list of test
        train_mean_ds: list. mean accuracy list of train
        train_ci_ds: list. confidence interval list of train
        
        
        
    NOTE 
        randomSplitCI could be the sub-route of this function
    """
    data_precentage = np.linspace(0.1, 1, 10, endpoint=True)
    
    ########### your code goes here ###########
    

    
    
    
    
    ###########         end         ###########
    return train_mean_ds, train_ci_ds, test_mean_ds, test_ci_ds

In [ ]:
train_mean_ds, train_ci_ds, test_mean_ds, test_ci_ds = dataSizeAC(data, 'LR', 3)

### (b)

In [ ]:
def dataSizePlot(datasize, train_mean_ds, train_ci_ds, test_mean_ds, test_ci_ds):
    # First illustrate basic pyplot interface, using defaults where possible.
    plt.figure()
    test_curve=plt.errorbar(datasize, test_mean_ds, color=sns.xkcd_rgb["pale red"], yerr=test_ci_ds)
    train_curve=plt.errorbar(datasize, train_mean_ds,color=sns.xkcd_rgb["denim blue"], yerr=train_ci_ds)
    plt.legend([test_curve, train_curve], ['Test', 'Train'])
    plt.xlabel('Data Percentage')
    plt.ylabel('Accuracy')
    plt.title("Accuracy vs Data Percentage")
    plt.show()

In [ ]:
dataSizePlot(np.linspace(0.1, 1, 10, endpoint=True),train_mean_ds, train_ci_ds, test_mean_ds, test_ci_ds)

## Q5

### (a)

### (b)

You should be able to get this answer easily by running dataPreprocessor fuction and randomSplitCI function

## Q6

### (a)